In [29]:
import pandas as pd
import numpy as np

df = pd.read_csv("data/devise.csv.gz")
print (df.head())
print(df.tail())

         date    exalus  excaus  exchus    exjpus  exszus
0  1999-01-31  1.582278  1.5194  8.2789  113.2900  1.3856
1  1999-02-28  1.562744  1.4977  8.2781  116.6684  1.4272
2  1999-03-31  1.585289  1.5176  8.2792  119.4730  1.4660
3  1999-04-30  1.557632  1.4881  8.2792  119.7723  1.4971
4  1999-05-31  1.508751  1.4611  8.2785  121.9995  1.5078
           date    exalus  excaus  exchus    exjpus  exszus
308  2024-09-30  1.477105  1.3546  7.0760  142.9540  0.8472
309  2024-10-31  1.492537  1.3757  7.0881  149.8909  0.8613
310  2024-11-30  1.531863  1.3973  7.2063  153.7126  0.8809
311  2024-12-31  1.578781  1.4247  7.2807  153.8143  0.8916
312  2025-01-31  1.604879  1.4389  7.2957  156.4819  0.9096


In [30]:
import pandas as pd
import re

# --- 1. Charger et préparer les taux BIS ---

url_cbpol = "https://data.bis.org/static/bulk/WS_CBPOL_csv_col.zip"
cb = pd.read_csv(url_cbpol, compression="zip", low_memory=False)

cb.columns = cb.columns.str.strip()

# Colonnes de dates (YYYY-MM-DD)
date_cols = [c for c in cb.columns if re.fullmatch(r"\d{4}-\d{2}-\d{2}", c)]
if not date_cols:
    raise ValueError("Aucune colonne de type YYYY-MM-DD trouvée dans le fichier BIS")

# Code pays propre (ex: 'US: United States' -> 'US')
cb["ref_code"] = cb["REF_AREA"].astype(str).str.split(":").str[0].str.strip()

# Pays et fréquence qui t'intéressent
codes_pays = ["US", "JP", "CH", "AU", "CN", "CA"]
cb_filt = cb[(cb["FREQ"] == "D") & (cb["ref_code"].isin(codes_pays))].copy()

# Wide -> long
cb_long = cb_filt.melt(
    id_vars=["ref_code"],
    value_vars=date_cols,
    var_name="date",
    value_name="rate"
)

cb_long["date"] = pd.to_datetime(cb_long["date"], errors="coerce")
cb_long = cb_long.dropna(subset=["date"])

# Période qui t'intéresse
debut = "1999-01-31"
fin   = "2025-01-31"
cb_long = cb_long[cb_long["date"].between(debut, fin)]

# Pivot: une colonne par pays
df_taux_daily = (
    cb_long
    .pivot(index="date", columns="ref_code", values="rate")
    .sort_index()
)

# --- 2. Passer en mensuel fin de mois (comme tes devises) ---

df_taux_m = df_taux_daily.resample("M").last()

# Renommer les colonnes si tu veux quelque chose de plus lisible
df_taux_m = df_taux_m.rename(columns={
    "US": "USA",
    "JP": "Japan",
    "CH": "Switzerland",
    "AU": "Australia",
    "CN": "China",
    "CA": "Canada"
})

# --- 3. Merge avec ton DataFrame de devises df ---

# Assure-toi que df['date'] est bien en datetime
df["date"] = pd.to_datetime(df["date"])

df_final = df.merge(
    df_taux_m,
    left_on="date",
    right_index=True,
    how="inner"         # ou "left" si tu veux garder toutes les dates FX même sans taux
)

# Optionnel: enlever les lignes avec taux manquants
df_final = df_final.dropna(subset=["USA", "Japan", "Switzerland", "Australia", "China", "Canada"])

print(df_final.head())
print(df_final.tail())


         date    exalus  excaus  exchus    exjpus  exszus  Australia  Canada  \
1  1999-02-28  1.562744  1.4977  8.2781  116.6684  1.4272       4.75    5.00   
19 2000-08-31  1.721763  1.4828  8.2796  108.0804  1.7149       6.25    5.75   
20 2000-09-30  1.811266  1.4864  8.2785  106.8375  1.7586       6.25    5.75   
21 2000-10-31  1.893939  1.5125  8.2785  108.4429  1.7745       6.25    5.75   
22 2000-11-30  1.916443  1.5426  8.2774  109.0095  1.7779       6.25    5.75   

    Switzerland  China  Japan   USA  
1           1.0   6.39   0.25  4.75  
19          3.5   5.85   0.25  6.50  
20          3.5   5.85   0.25  6.50  
21          3.5   5.85   0.25  6.50  
22          3.5   5.85   0.25  6.50  
          date    exalus  excaus  exchus    exjpus  exszus  Australia  Canada  \
308 2024-09-30  1.477105  1.3546  7.0760  142.9540  0.8472       4.35    4.25   
309 2024-10-31  1.492537  1.3757  7.0881  149.8909  0.8613       4.35    3.75   
310 2024-11-30  1.531863  1.3973  7.2063  153.71

C:\Users\masth\AppData\Local\Temp\ipykernel_18236\3035959793.py:48: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_taux_m = df_taux_daily.resample("M").last()


In [31]:
import pandas as pd
from fredapi import Fred

fred = Fred(api_key="cb55babd6bab68262e9308fe1bf27976")

fred_codes = {
    "USA": "CPIAUCSL",
    "Switzerland": "CHECPIALLMINMEI",
    "Japan": "JPNCPIALLMINMEI",
    "Canada": "CPALTT01CAM659N",
    "Australia": "CCRETT01AUM661N",
    "China": "CPALTT01CNM659N"
}

# ---------------------------
# 2. Récupération inflation
# ---------------------------
df_infl = pd.DataFrame()
ok_codes = []
bad_codes = []

for country, code in fred_codes.items():
    try:
        series = fred.get_series(code)
        series.name = country
        df_infl[country] = series
        ok_codes.append((country, code))
    except Exception as e:
        print(f"ÉCHEC pour {country} avec code {code} : {e}")
        bad_codes.append((country, code))

print("Séries téléchargées avec succès :")
for c, code in ok_codes:
    print(f"  {c}: {code}")

# ---------------------------
# 3. Nettoyage index / tri
# ---------------------------
df_infl.index = pd.to_datetime(df_infl.index)
df_infl = df_infl.sort_index()

# ---------------------------
# 4. Inflation YoY (%)
# ---------------------------
df_infl_yoy = df_infl.pct_change(12) * 100

# ---------------------------
# 5. Passage mensuel fin de mois
# ---------------------------
df_infl_m = df_infl_yoy.resample("M").last()

# Renommer colonnes pour éviter confusion avec taux
df_infl_m = df_infl_m.add_suffix("_infl")

# ---------------------------
# 6. Merge avec ton df_final
#    (df_final contient déjà FX + taux)
# ---------------------------

# IMPORTANT : s'assurer que df_final["date"] existe et est bien datetime
df_final["date"] = pd.to_datetime(df_final["date"])

df_full = df_final.merge(
    df_infl_m,
    left_on="date",
    right_index=True,
    how="inner"
)

# ---------------------------
# 7. Aperçu
# ---------------------------
print(df_full.head())
print(df_full.tail())
print(df_full.columns)



Séries téléchargées avec succès :
  USA: CPIAUCSL
  Switzerland: CHECPIALLMINMEI
  Japan: JPNCPIALLMINMEI
  Canada: CPALTT01CAM659N
  Australia: CCRETT01AUM661N
  China: CPALTT01CNM659N
         date    exalus  excaus  exchus    exjpus  exszus  Australia  Canada  \
1  1999-02-28  1.562744  1.4977  8.2781  116.6684  1.4272       4.75    5.00   
19 2000-08-31  1.721763  1.4828  8.2796  108.0804  1.7149       6.25    5.75   
20 2000-09-30  1.811266  1.4864  8.2785  106.8375  1.7586       6.25    5.75   
21 2000-10-31  1.893939  1.5125  8.2785  108.4429  1.7745       6.25    5.75   
22 2000-11-30  1.916443  1.5426  8.2774  109.0095  1.7779       6.25    5.75   

    Switzerland  China  Japan   USA  USA_infl  Switzerland_infl  Japan_infl  \
1           1.0   6.39   0.25  4.75  1.666667          0.288021   -0.102242   
19          3.5   5.85   0.25  6.50  3.351287          1.124746   -0.510720   
20          3.5   5.85   0.25  6.50  3.456496          1.439946   -0.916503   
21          3.5  

C:\Users\masth\AppData\Local\Temp\ipykernel_18236\3339177239.py:45: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_infl_yoy = df_infl.pct_change(12) * 100
C:\Users\masth\AppData\Local\Temp\ipykernel_18236\3339177239.py:50: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_infl_m = df_infl_yoy.resample("M").last()


In [50]:
import numpy as np

# Repartir du df_final propre (avant les transformations)
df_full = df_final.copy()

# Date en index
df_full = df_full.sort_values("date").set_index("date")

# Log-returns
fx_cols = ["exalus", "excaus", "exchus", "exjpus", "exszus"]
for col in fx_cols:
    df_full[f"logret_{col}"] = np.log(df_full[col]) - np.log(df_full[col].shift(1))

df_full = df_full.dropna()

# Carry
rate_cols = ["USA", "Japan", "Switzerland", "Australia", "China", "Canada"]
for c in rate_cols:
    if c != "USA":
        df_full[f"carry_{c}_USD"] = df_full[c] - df_full["USA"]

#VIX
# --- 1. Télécharger le VIX depuis FRED ---
vix = fred.get_series("VIXCLS")
vix.index = pd.to_datetime(vix.index)
vix.name = "VIX"

# --- 2. Transformer en mensuel fin de mois ---
vix_m = vix.resample("M").last()

# --- 3. Fusionner avec df_full (ton DataFrame complet) ---
df_full = df_full.merge(
    vix_m,
    left_on="date",
    right_index=True,
    how="left")

# --- 4. Standardiser le VIX ---
df_full["VIX_std"] = (df_full["VIX"] - df_full["VIX"].mean()) / df_full["VIX"].std()

# Standardisation (colonnes de base seulement)
base_cols = fx_cols + rate_cols + [c + "_infl" for c in rate_cols if c + "_infl" in df_full.columns]
for col in base_cols:
    if col in df_full.columns:
        df_full[col + "_std"] = (df_full[col] - df_full[col].mean()) / df_full[col].std()

print(df_full.head())

              exalus  excaus  exchus    exjpus  exszus  Australia  Canada  \
date                                                                        
2000-08-31  1.721763  1.4828  8.2796  108.0804  1.7149       6.25    5.75   
2000-09-30  1.811266  1.4864  8.2785  106.8375  1.7586       6.25    5.75   
2000-10-31  1.893939  1.5125  8.2785  108.4429  1.7745       6.25    5.75   
2000-11-30  1.916443  1.5426  8.2774  109.0095  1.7779       6.25    5.75   
2000-12-31  1.829491  1.5219  8.2771  112.2090  1.6855       6.25    5.75   

            Switzerland  China  Japan  ...  excaus_std  exchus_std  \
date                                   ...                           
2000-08-31          3.5   5.85   0.25  ...    1.752775    2.646865   
2000-09-30          3.5   5.85   0.25  ...    1.776051    2.644704   
2000-10-31          3.5   5.85   0.25  ...    1.944805    2.644704   
2000-11-30          3.5   5.85   0.25  ...    2.139422    2.642543   
2000-12-31          3.5   5.85   0.25  .

C:\Users\masth\AppData\Local\Temp\ipykernel_18236\332539038.py:29: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  vix_m = vix.resample("M").last()
